In [1]:
!pip install nltk
!pip install openpyxl

In [2]:
import pandas as pd
import re
import nltk
from nltk.util import ngrams
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=bfda66bc5d67d7559cb80e875d7e60d781c2f585607317bfaa6d2e5b741b3734
  Stored in directory: /home/codespace/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [29]:
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDiA
from nltk.tokenize import casual_tokenize
from langdetect import detect, detect_langs
from nltk.corpus import stopwords

In [27]:
df=pd.read_excel('/workspaces/BMIG60303-assignments/All_Articles_Excel_Dec2019July2020.xlsx')

In [30]:
df.head()

,Date Added,Author,Title,Abstract,Year,Journal/Publisher,Volume,Issue,Pages,Accession Number,...,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93
0,2020-07-31,"Zwerling, A",Understanding spending trends for tuberculosis,Total out-of-pocket spending decreased over th...,2020,Lancet Infectious Diseases,20,8,879-880,2428388309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-07-31,"Zucoloto, GM, Pedro; Porto, Patricia",A propriedade industrial pode limitar o combat...,Esta nota técnica apresenta alguns dos pontos ...,2020,NaN,NaN,Issue,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-07-31,"Zimmer, MAZ, Anne K.; Weißer, Christian W.; Vo...",Hypernatremia—A Manifestation of COVID-19: A C...,We report for the first time therapy-resistant...,2020,A&A Practice,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-07-31,"Zignale, M",The experienced space between mobility and Cov...,If it is true that lived space represents our ...,2020,Documenti Geografici,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-07-31,"Zhu, XC, Bodong; Avadhanam, Rukmini Manasa; Sh...",Reading and connecting: using social annotatio...,Purpose - The COM-19 pandemic has forced many ...,2020,Information and Learning Sciences,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df.columns

Index(['Date Added', 'Author', 'Title', 'Abstract', 'Year',
       'Journal/Publisher', 'Volume', 'Issue', 'Pages', 'Accession Number',
       'DOI', 'URL', 'Name of Database', 'Database Provider', 'Language',
       'Keywords', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51',
       'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55',
       'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59',
       'Unnamed: 60', 'Unnamed: 61', 'Unn

In [32]:
df = df[['Date Added', 'Author', 'Title', 'Abstract', 'Year']]
df.dropna(subset=['Abstract'], how='all', inplace=True)
df['Abstract'] = df['Abstract'].str.lower()

In [33]:
df.head()

,Date Added,Author,Title,Abstract,Year
0,2020-07-31,"Zwerling, A",Understanding spending trends for tuberculosis,total out-of-pocket spending decreased over th...,2020
1,2020-07-31,"Zucoloto, GM, Pedro; Porto, Patricia",A propriedade industrial pode limitar o combat...,esta nota técnica apresenta alguns dos pontos ...,2020
2,2020-07-31,"Zimmer, MAZ, Anne K.; Weißer, Christian W.; Vo...",Hypernatremia—A Manifestation of COVID-19: A C...,we report for the first time therapy-resistant...,2020
3,2020-07-31,"Zignale, M",The experienced space between mobility and Cov...,if it is true that lived space represents our ...,2020
4,2020-07-31,"Zhu, XC, Bodong; Avadhanam, Rukmini Manasa; Sh...",Reading and connecting: using social annotatio...,purpose - the com-19 pandemic has forced many ...,2020


In [34]:
df.iloc[0]['Abstract']

"total out-of-pocket spending decreased over the same period; however, although the authors captured direct out-of-pocket spending on medical expenses, they did not include non-medical costs including loss of income, transport, and indirect economic costs due to tuberculosis (many of which are now being collected through who patient cost surveys) in their analysis. the authors' findings show that three countries with strong private sectors—democratic republic of the congo, nigeria, and pakistan—have out-of-pocket medical expenses as the primary source of tuberculosis spending. [...]trends over time and across countries can be used to monitor fluctuations in total tuberculosis spending and assess needs across regions."

In [36]:
df = df[~df.Abstract.isna()]


def is_english(text, thresh = .9):
    try:
        langs = detect_langs(text)
        return {l.lang : l.prob for l in langs}['en'] > thresh
    except:
        return False

    
adf2 = df.sample(5000)
adf2['English'] = adf2.Abstract.apply(is_english)
adf2 = adf2[adf2.English]

In [37]:
adf2

,Date Added,Author,Title,Abstract,Year,English
67597,2020-04-07,"Gursel, MG, Ihsan",Is Global BCG Vaccination Coverage Relevant To...,the lower than expected number of sars-cov-2 c...,2020,True
66011,2020-04-13,"Han, YYZ, M. R.; Shi, Y.; Song, Z. H.; Zhou, S...",Application of integrative medicine protocols ...,since the severe acute respiratory syndrome co...,2020,True
11294,2020-07-14,"Zhu, ST, J.; Gao, H.; He, D.","Age, source, and future risk of COVID-19 infec...","objective: to explore and compare the age, sou...",2020,True
28591,2020-06-15,"Allali, GM, C.; Grosgurin, O.; Morelot-Panzini...",Dyspnea: the vanished warning symptom of COVID...,"since december 2019, the severe acute respirat...",2020,True
60306,2020-04-24,"Stedman, MD, Mark; Anderson, Simon G.; Lunt, M...",A phased approach to unlocking during the COVI...,with the covid-19 pandemic leading to radical ...,2020,True
...,...,...,...,...,...,...
25174,2020-06-22,"Lanjiwar, MNS, Ananthu; Garg, Rakesh; Srivasta...",Neurological Manifestations in Mild SARS-CoV-2...,background: the outbreak of novel corona virus...,2020,True
22640,2020-06-24,"Zhou, YJZ, K. I.; Wang, X. B.; Sun, Q. F.; Pan...",Metabolic associated fatty liver disease is as...,the corona virus disease 2019 (covid-19) pande...,2020,True
75834,2020-03-20,"Fang, YN, Y.; Penny, M.",Transmission dynamics of the COVID-19 outbreak...,using the parameterized susceptible-exposed-in...,2020,True
25482,2020-06-22,"Fosbol, ELB, J. H.; Ostergaard, L.; Andersson,...",Association of Angiotensin-Converting Enzyme I...,importance: it has been hypothesized that angi...,2020,True


In [25]:
df

,Date Added,Author,Title,Abstract,Year,English


In [23]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
def part01(df):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['Abstract'])
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

    return tfidf_df

In [25]:
df_tfidf = part01(df.sample(frac=0.1, random_state=42))

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [26]:
df_tfidf.head()

,00,000,0000,00001,000022583,0000471,00006,00007,0001,00015,...,黄芩可通过多途径,黄芩改善covid,黄芩核心靶标,黄芩素,黄芩药对具有抗炎,黄芩药对改善covid,黄芩药对改善新型冠状病毒,黄芩药对的临床应用提供参考,黄芪,鼻塞
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.029039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
import pandas as pd

def part02(df_vectors):
    top_words_per_abstract = []
    for i in range(min(10, df_vectors.shape[0])):
        doc_vector = df_vectors.iloc[i]
        top_words = doc_vector.sort_values(ascending=False).head(5)
        top_words_list = [(word, score) for word, score in top_words.items()]
        top_words_per_abstract.append(top_words_list)

    return top_words_per_abstract


In [28]:
top_words = part02(df_tfidf)

In [29]:
top_words

[[('本文分析武汉某新建治疗新型冠状病毒肺炎医院院所配备的药品目录和四味专有中药组方', 0.3779644730092272),
  ('提高临床治疗效果', 0.3779644730092272),
  ('分布', 0.3779644730092272),
  ('代谢和排泄四个方面总结了中西药相互作用的机制和用药监护要点', 0.3779644730092272),
  ('提高用药安全性具有重要意义', 0.3779644730092272)],
 [('deprivation', 0.41341520534712756),
  ('chennai', 0.351177686586386),
  ('megacity', 0.30728047576308776),
  ('nbr', 0.27614158078264833),
  ('wards', 0.2150532548816092)],
 [('coefficients', 0.2721853810972516),
  ('data', 0.2179481927069262),
  ('the', 0.1984725447774574),
  ('set', 0.17672111264349175),
  ('fit', 0.17585399792423398)],
 [('the', 0.2494100249300536),
  ('technology', 0.23192688034845169),
  ('synchronicity', 0.21967073876524978),
  ('auspicious', 0.21967073876524978),
  ('1950s', 0.21967073876524978)],
 [('同时', 0.2901346225257028),
  ('cov', 0.23550810382798631),
  ('的来源', 0.15750774213227178),
  ('没有证据显示宠物会感染新型冠状病毒', 0.15750774213227178),
  ('彰显了宠物与人的亲密程度及其家庭地位之高', 0.15750774213227178)],
 [('resection', 0.26304733570668354),
  ('surgi

In [27]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def part03(df_vectors, query_string):
    vectorizer = TfidfVectorizer(vocabulary=df_vectors.columns)
    query_vector = vectorizer.fit_transform([query_string])
    cosine_similarities = cosine_similarity(query_vector, df_vectors).flatten()
    top_5_indices = cosine_similarities.argsort()[-5:][::-1]

    return top_5_indices


In [28]:
query_string = "machine learning"
op_docs = part03(df_tfidf, query_string)

In [29]:
op_docs

array([4106, 4354, 3413, 3113, 4658])

In [30]:
df.iloc[[4106, 4354, 3413, 3113, 4658]]['Abstract']

6092    over the past years, several zoonotic viruses ...
6420    coronavirus disease 2019, i.e. covid-19, start...
5081    two recent lancet and lancet oncology papers r...
4625    whether weather plays a part in the transmissi...
6737    武 汉 新 型 冠 状 病 毒 肺 炎 是 指 由 新 型 冠 状 病 毒 (2019-nc...
Name: Abstract, dtype: object

# Part 1

## Test Regex

In [9]:
pattern = r'\bItaly|Italian\b'
re.findall(pattern, 'Italy ITALY ITALIAN Italian italy italian ItaLian iTalY', re.IGNORECASE)

['Italy',
 'ITALY',
 'ITALIAN',
 'Italian',
 'italy',
 'italian',
 'ItaLian',
 'iTalY']

In [10]:
matches_l = []
for index, row in df.iterrows():
    abstract = row['Abstract']
    if isinstance(abstract, str):
        matches = re.findall(pattern, abstract, re.IGNORECASE)
        match_count = len(matches)
        if match_count > 0:
            matches_l.append((matches, match_count, abstract))
top_matches = sorted(matches_l, key=lambda x: x[1], reverse=True)[:10]

In [11]:
for i in top_matches:
    print(i[0])
    print(i[1])
    print(i[2])
    

['Italy', 'Italy', 'Italy', 'Italian', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italian', 'Italy', 'Italy']
12
Italy is currently experiencing an epidemic of COVID-19 which emerged in the Lombardy region . During the interval between February 25-29, 2020, we identified 46 cases of COVID-19 reported in 21 countries in Europe, Africa, North America, and South America which were either in individuals with recent travel from Italy, or who had presumed infection by a traveler from Italy 2. In six cases, in four of the affected countries (Switzerland, France, Austria, Croatia), land travel was a likely route of introduction, or was documented to have been the route of introduction. We used air travel volume between Italian cities and cities in other countries as an index of connectedness, using data available from the International Air Transport Association (IATA) for February 2015 (2.61 million total departing international air passengers from Italy). We used the methods of Fraser et a

# Part 2
## Test tokenizer

In [12]:
from nltk.tokenize import wordpunct_tokenize
text = 'This is a Italy of the ITALY. ITALIAN Italian in the realm. That; right there is italy "italian" ItaLian iTalY.'
tokens = wordpunct_tokenize(text)
print(tokens)

['This', 'is', 'a', 'Italy', 'of', 'the', 'ITALY', '.', 'ITALIAN', 'Italian', 'in', 'the', 'realm', '.', 'That', ';', 'right', 'there', 'is', 'italy', '"', 'italian', '"', 'ItaLian', 'iTalY', '.']


In [13]:
bigrams = Counter()
pattern = re.compile(r'\b(Italy|Italian)\b', re.IGNORECASE)
for index, row in df.iterrows():
    abstract = row['Abstract']
    if isinstance(abstract, str):
        tokens = wordpunct_tokenize(abstract)
        bigrams_list = list(ngrams(tokens, 2))
        re_search = [bigram for bigram in bigrams_list if pattern.search(bigram[0])]
        bigrams.update(re_search)

In [14]:
common_bigrams = bigrams.most_common(20)
for i in common_bigrams:
    print(str(i[0])+"\t"+ str(i[1]))

('Italy', ',')	703
('Italy', '.')	282
('Italy', 'and')	274
('Italy', 'is')	94
('Italy', '(')	79
('Italian', 'regions')	77
('Italy', 'has')	69
('Italian', 'Society')	57
('Italy', 'was')	56
('Italy', ')')	54
('Italian', 'population')	34
('Italian', 'government')	33
('Italian', 'National')	28
('Italian', 'Ministry')	28
('Italy', '),')	27
('Italy', 'in')	26
('Italian', 'Government')	23
('Italian', 'COVID')	19
('Italy', 'on')	18
('Italy', 'with')	18


# Part 3

## Test tokenizer

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [17]:
text = 'This is a Italy of the ITALY. ITALIAN Italian in the realm. That; right there is italy "italian" ItaLian iTalY.'
tokens = wordpunct_tokenize(text)
cleaned_tokens = [token for token in tokens if token.isalpha() and token.lower() not in stop_words]
cleaned_tokens

['Italy',
 'ITALY',
 'ITALIAN',
 'Italian',
 'realm',
 'right',
 'italy',
 'italian',
 'ItaLian',
 'iTalY']

In [18]:
bigrams = Counter()
pattern = re.compile(r'\b(Italy|Italian)\b', re.IGNORECASE)
for index, row in df.iterrows():
    abstract = row['Abstract']
    if isinstance(abstract, str):
        tokens = wordpunct_tokenize(abstract)
        cleaned_tokens = [token for token in tokens if token.isalnum() and token.lower() not in stop_words]
        bigrams_list = list(ngrams(cleaned_tokens, 2))
        re_search = [bigram for bigram in bigrams_list if pattern.search(bigram[0])]
        bigrams.update(re_search)

In [19]:
common_bigrams = bigrams.most_common(20)
common_bigrams

[(('Italy', 'Spain'), 177),
 (('Italian', 'regions'), 77),
 (('Italian', 'Society'), 57),
 (('Italy', 'one'), 56),
 (('Italy', 'first'), 52),
 (('Italy', 'Iran'), 49),
 (('Italian', 'population'), 34),
 (('Italian', 'government'), 33),
 (('Italy', 'Germany'), 29),
 (('Italy', 'France'), 29),
 (('Italian', 'National'), 28),
 (('Italy', 'China'), 28),
 (('Italian', 'Ministry'), 28),
 (('Italy', 'South'), 26),
 (('Italian', 'Government'), 23),
 (('Italy', 'USA'), 23),
 (('Italy', 'March'), 22),
 (('Italy', 'United'), 20),
 (('Italian', 'COVID'), 19),
 (('Italian', 'provinces'), 17)]

In [3]:
df = pd.read_csv('1557tweets.csv')

In [4]:
df.head()

,text
0,Kodak Black - Codeine Dreaming &gt;&gt;
1,"give me some morphine, is there any more to do?"
2,"just like nicotine,\nheroin, morphine"
3,Codeine hella calling my name 🙄
4,Minus how much coke you youngins be on. Like y...


In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...


True

In [7]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

def part01():
    synset_consume = wn.synset('consume.v.02')
    hyponyms = synset_consume.closure(lambda s: s.hyponyms())
    return set(hyponyms)
result = part01()
print(result)
print(len(result))


{Synset('gluttonize.v.01'), Synset('sip.v.01'), Synset('receive.v.12'), Synset('pub-crawl.v.01'), Synset('tuck_in.v.01'), Synset('hit.v.15'), Synset('wash_down.v.01'), Synset('ruminate.v.01'), Synset('pop.v.11'), Synset('guzzle.v.01'), Synset('fare.v.02'), Synset('dunk.v.03'), Synset('wine_and_dine.v.01'), Synset('eat_up.v.01'), Synset('sup.v.01'), Synset('eat_out.v.01'), Synset('toss_off.v.02'), Synset('crop.v.05'), Synset('eat.v.01'), Synset('raven.v.02'), Synset('lunch.v.01'), Synset('inject.v.04'), Synset('drop.v.17'), Synset('guggle.v.03'), Synset('eat.v.02'), Synset('tank.v.02'), Synset('use.v.02'), Synset('mess.v.01'), Synset('chain-smoke.v.01'), Synset('port.v.07'), Synset('slurp.v.01'), Synset('nibble.v.03'), Synset('fill_up.v.04'), Synset('mainline.v.01'), Synset('degust.v.01'), Synset('partake.v.03'), Synset('pitch_in.v.01'), Synset('board.v.03'), Synset('break_bread.v.01'), Synset('carry.v.33'), Synset('take_out.v.12'), Synset('swill.v.02'), Synset('picnic.v.01'), Synset('s

In [9]:
def part02(synset_set):
    lemmas = {lemma for synset in synset_set for lemma in synset.lemma_names()}
    filtered_lemmas = {lemma for lemma in lemmas if '_' not in lemma}
    return filtered_lemmas

synsets = part01()
lemmas = part02(synsets)
print(lemmas)
print(len(lemmas))


{'bolt', 'gulp', 'souse', 'tope', 'fress', 'pall', 'guzzle', 'dunk', 'snack', 'whiff', 'pick', 'gurgle', 'inject', 'gobble', 'hit', 'englut', 'guggle', 'feast', 'drop', 'soak', 'dip', 'finish', 'swill', 'partake', 'forage', 'hold', 'stuff', 'victual', 'gormandise', 'breakfast', 'gluttonise', 'use', 'tipple', 'overeat', 'inhale', 'wine', 'gorge', 'dope', 'satiate', 'ruminate', 'port', 'habituate', 'suckle', 'ingurgitate', 'gluttonize', 'gourmandize', 'lunch', 'claret', 'lap', 'dine', 'drink', 'degust', 'cannibalise', 'inebriate', 'feed', 'imbibe', 'smoke', 'cannibalize', 'carry', 'sample', 'sup', 'board', 'graze', 'kill', 'browse', 'prey', 'drug', 'junket', 'nosh', 'swallow', 'slurp', 'piece', 'fuddle', 'pig', 'quaff', 'overindulge', 'brunch', 'suck', 'predate', 'free-base', 'binge', 'base', 'peck', 'eat', 'gormandize', 'banquet', 'chain-smoke', 'sip', 'engorge', 'fare', 'down', 'try', 'receive', 'cloy', 'mess', 'crop', 'picnic', 'glut', 'pub-crawl', 'overgorge', 'wolf', 'snort', 'tank'

In [12]:
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.7/31.7 MB 90.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.0/865.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 79.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.0
    Uninstalling numpy-2.1.0:
      Successfully uninstalled numpy-2.1.0


In [23]:
import spacy
from nltk.corpus import stopwords
from collections import Counter
import string

import en_core_web_sm
nlp = en_core_web_sm.load()
def part03(texts, consume_terms):
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    bigram_counter = Counter()
    for text in texts:
        doc = nlp(text)
        tokens = [
            token.lemma_.lower() for token in doc
            if token.lemma_.lower() not in stop_words and token.lemma_ not in punctuation and not token.is_punct
        ]
        for i in range(len(tokens) - 1):
            first_word = tokens[i]
            second_word = tokens[i + 1]
            if first_word in consume_terms:
                bigram_counter[(first_word, second_word)] += 1
    return bigram_counter

# Example usage:
# Load the tweets from the CSV file
df = pd.read_csv('1157tweets.csv')

# Get the list of consume terms by calling part01 and part02
consume_terms = part02(part01())

# Call part03 to process the tweets and find relevant bigrams
res = part03(df['text'], consume_terms)

# Print the most common bigrams
print(res.most_common(10))


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [17]:
! --yes -r
! --yes -r


Usage:   
  pip uninstall [options] <package> ...
  pip uninstall [options] -r <requirements file> ...

-r option requires 1 argument

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --yes


In [24]:
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

NameError: name 'spacy' is not defined

In [25]:
import spacy

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [1]:
import pandas as pd
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
# Step 1: Load data (Assume you load your dataset into a DataFrame)
data = pd.read_excel('/workspaces/BMIG60303-assignments/All_Articles_Excel_Dec2019July2020.xlsx')
df = data[['Abstract']].dropna()  # Filter out missing abstracts

# Step 2: Ensure all abstracts are strings and clean data
df['Abstract'] = df['Abstract'].astype(str)  # Convert all entries to strings

# Step 3: Detect English abstracts
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

df['English'] = df['Abstract'].apply(is_english)

# Step 4: Extract 5000 random abstracts
random_abstracts_df = df[df['English']].sample(n=5000, random_state=0)  # Only English abstracts

# Step 5: Vectorize abstracts using CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
X = vectorizer.fit_transform(random_abstracts_df['Abstract'])

# Step 6: Fit LDA model with 10 topics
lda = LatentDirichletAllocation(n_components=10, random_state=0)
lda.fit(X)

# Step 7: Function to display topics
def get_topic_words(lda_model, feature_names, n_top_words=10):
    topics = {}
    for topic_idx, topic in enumerate(lda_model.components_):
        topic_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topics[f"Topic {topic_idx+1}"] = topic_words
    return topics

# Get the top words for each topic
feature_names = vectorizer.get_feature_names_out()
topics = get_topic_words(lda, feature_names)
for topic, words in topics.items():
    print(f'{topic}: {", ".join(words)}')

# Step 8: Visualize with PyLDAVis (Updated for newer versions)

# Extract necessary values for pyLDAvis
vocab = vectorizer.get_feature_names_out()  # Vocabulary
term_frequency = X.sum(axis=0).A1  # Term frequencies
doc_lengths = X.sum(axis=1).A1  # Document lengths (needed for visualization)

# Prepare the visualization
lda_vis = pyLDAvis.prepare(
    topic_term_dists=lda.components_,
    doc_topic_dists=lda.transform(X),
    doc_lengths=doc_lengths,
    vocab=vocab,
    term_frequency=term_frequency
)

# Save visualization as HTML
pyLDAvis.save_html(lda_vis, 'lda_part1.html')


Topic 1: health, coronavirus, use, research, cost, human, public, application, invention, telemedicine
Topic 2: registered, study, research, obtained, irb, guidelines, relevant, id, competing, reporting
Topic 3: cov, sars, protein, virus, mers, binding, coronavirus, spike, cell, human
Topic 4: covid, 19, pandemic, health, care, medical, patients, healthcare, risk, social
Topic 5: sars, 19, cov, covid, coronavirus, disease, respiratory, infection, 2019, treatment
Topic 6: study, 19, covid, research, patients, 95, amp, data, ci, trial
Topic 7: 19, covid, health, cases, pandemic, 2020, countries, data, measures, public
Topic 8: patients, 19, covid, disease, severe, risk, clinical, studies, mortality, infection
Topic 9: patients, 19, covid, cases, results, sars, cov, positive, clinical, pcr
Topic 10: mask, masks, transmission, virus, use, n95, viral, respiratory, wearing, protective


/home/codespace/.local/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=7272) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/codespace/.local/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=7272) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/codespace/.local/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=7272) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()


In [1]:
# Import necessary libraries
import pandas as pd
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
import numpy as np

# Helper function to detect if the text is in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# Function to fit and display topics from an LDA model
def fit_and_print_lda(X, n_topics, vectorizer, n_top_words=10):
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
    lda.fit(X)

    feature_names = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(lda.components_):
        print(f"\nTopic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Load dataset (replace with actual dataset path)
df = pd.read_excel('/workspaces/BMIG60303-assignments/All_Articles_Excel_Dec2019July2020.xlsx')

# Step 1: Filter out non-English abstracts
df['English'] = df['Abstract'].apply(is_english)
df = df[df['English']]  # Filter to include only English abstracts

# Step 2: Extract 5000 random abstracts
random_abstracts_df = df.sample(n=5000, random_state=0)

# Define the custom tokenizer
tokenizer = TreebankWordTokenizer()

# Define custom stopwords (can be expanded)
custom_stopwords = set(stopwords.words('english')).union({'covid', 'coronavirus', 'sars', 'cov', '19', 'doi'})
custom_stopwords_list = list(custom_stopwords)  # Convert to list for TfidfVectorizer

# Step 3: Experimenting with different configurations

# 1. Use TfidfVectorizer with custom tokenization and stopwords
tfidf_vectorizer_custom_stopwords = TfidfVectorizer(
    max_df=0.95,
    min_df=5,
    stop_words=custom_stopwords_list,  # Now the stopwords are passed as a list
    tokenizer=tokenizer.tokenize, 
    max_features=5000
)

# Fit and transform the abstracts
X_tfidf_custom_stopwords = tfidf_vectorizer_custom_stopwords.fit_transform(random_abstracts_df['Abstract'])

# Apply LDA with 10 topics and print results
print("LDA with 10 Topics (Custom Stopwords & Tokenizer):")
fit_and_print_lda(X_tfidf_custom_stopwords, n_topics=10, vectorizer=tfidf_vectorizer_custom_stopwords)

# 2. Experiment with fewer topics (e.g., 5 topics)
print("\nLDA with 5 Topics (Custom Stopwords & Tokenizer):")
fit_and_print_lda(X_tfidf_custom_stopwords, n_topics=5, vectorizer=tfidf_vectorizer_custom_stopwords)

# 3. Experiment with more topics (e.g., 15 topics)
print("\nLDA with 15 Topics (Custom Stopwords & Tokenizer):")
fit_and_print_lda(X_tfidf_custom_stopwords, n_topics=15, vectorizer=tfidf_vectorizer_custom_stopwords)

# 4. Experiment with different number of features
tfidf_vectorizer_fewer_features = TfidfVectorizer(
    max_df=0.95,
    min_df=5,
    stop_words=custom_stopwords_list,
    tokenizer=tokenizer.tokenize,
    max_features=2000  # Reduce the number of features
)

X_tfidf_fewer_features = tfidf_vectorizer_fewer_features.fit_transform(random_abstracts_df['Abstract'])

# LDA with 10 topics and fewer features
print("\nLDA with 10 Topics (Fewer Features):")
fit_and_print_lda(X_tfidf_fewer_features, n_topics=10, vectorizer=tfidf_vectorizer_fewer_features)

# 5. Experiment without stopwords
tfidf_vectorizer_no_stopwords = TfidfVectorizer(
    max_df=0.95,
    min_df=5,
    tokenizer=tokenizer.tokenize,
    max_features=5000
)

X_tfidf_no_stopwords = tfidf_vectorizer_no_stopwords.fit_transform(random_abstracts_df['Abstract'])

# LDA without stopwords and 10 topics
print("\nLDA with 10 Topics (No Stopwords):")
fit_and_print_lda(X_tfidf_no_stopwords, n_topics=10, vectorizer=tfidf_vectorizer_no_stopwords)

# Step 4: Summary of the changes in topics

# 1. Default stopwords and default tokenization
print("\nLDA with 10 Topics (Default Stopwords and Tokenizer):")
vectorizer_default = TfidfVectorizer(max_df=0.95, min_df=5, stop_words='english', max_features=5000)
X_default = vectorizer_default.fit_transform(random_abstracts_df['Abstract'])
fit_and_print_lda(X_default, n_topics=10, vectorizer=vectorizer_default)

# 2. Adding a different tokenizer and adjusting stopwords
# Already done above with custom tokenization and custom stopwords

# 3. Modifying the number of topics
# Done with n_components as 5, 10, and 15 in experiments

# You can add more experiments to see the impact of features, topics, or vectorization settings

# Final Conclusions can be drawn based on the outputs, themes, and coherence of the topics.


/home/codespace/.local/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


LDA with 10 Topics (Custom Stopwords & Tokenizer):

Topic 1:
hybrid parallel education. videos face-to-face advances sports directions strategies. describing

Topic 2:
mpro anal. quercetin voice beliefs shared wind opening promise remove

Topic 3:
, health pandemic covid-19 public social ; response care crisis

Topic 4:
migrants youth updates correction humanity emerge respect following gates article

Topic 5:
pad gi peer crime australia − australian eastern danger manual

Topic 6:
, ) ( % : covid-19 patients study ; registered

Topic 7:
copyright users abstract print [ ] download abstracts holder emailed

Topic 8:
• t-cell apps ssa hyperglycemia interview log dangerous dealing comply

Topic 9:
conjunctival charged hace2 div built instruments mpro publishing charge pandemia

Topic 10:
, covid-19 patients ) ( disease . respiratory care pandemic

LDA with 5 Topics (Custom Stopwords & Tokenizer):

Topic 1:
br div layer < neurosurgical table manual exercise top complete

Topic 2:
, sars-co

In [3]:
import glob
from random import shuffle
import os
import re
import tarfile
import tqdm
import pandas as pd

import numpy as np  # Keras takes care of most of this but it likes to see Numpy arrays
from keras.preprocessing import sequence    # A helper module to handle padding input
from keras.models import Sequential         # The base keras Neural Network model
from keras.layers import Dense, Dropout, Activation   # The layer objects we will pile into the model
from keras.layers import Conv1D, GlobalMaxPooling1D

from nltk.tokenize import TreebankWordTokenizer
from gensim.models import KeyedVectors

# pull in a dataframe of ade texts and labels
ade_df = pd.read_parquet("hf://datasets/ade-benchmark-corpus/ade_corpus_v2/Ade_corpus_v2_classification/train-00000-of-00001.parquet")
    
# Google pre-trained vectors, available here:
# https://code.google.com/archive/p/word2vec/
googlevecs = 'GoogleNews-vectors-negative300.bin.gz'
word_vectors = KeyedVectors.load_word2vec_format(googlevecs, binary=True, limit=200000)

def pre_process_data(filepath):
    """
    This is dependent on your training data source but we will try to generalize it as best as possible.
    """
    positive_path = os.path.join(filepath, 'pos')
    negative_path = os.path.join(filepath, 'neg')

    pos_label = 1
    neg_label = 0

    dataset = []

    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((pos_label, f.read()))

    for filename in glob.glob(os.path.join(negative_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((neg_label, f.read()))

    shuffle(dataset)

    return dataset







def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()

    vectorized_data = []
    expected = []

    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])

        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])
            except KeyError:
#                print(f"not found: {token}")
                pass
             
        vectorized_data.append(sample_vecs)
    return vectorized_data



def collect_expected(dataset):
    """ Peel of the target values from the dataset """
    expected = []
    for sample in dataset:
        expected.append(sample[0])
    return expected

"""
Following the previous example, the Dataset format is a list of tuples,
with label as the first element and text as the second element.
Here we re-shape the ADE dataframe to fit this format.
"""
dataset = [(x[2], x[1]) for x in ade_df.itertuples()]

"""
Note: as discussed in class on Oct 17, many terms relevant to identifying adverse drug events,
such a drug names and the terms for particular symptoms/problems, may not be available in the
word2vec embeddings generated from Google news.

One solution would be to use another set of pre-computed vectors.
If there's not a good option for this type of text, we also might look into building our own.
"""
vectorized_data = tokenize_and_vectorize(dataset)

expected = collect_expected(dataset)

split_point = int(len(vectorized_data) * .8)

x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]



"""
Note: as discussed in class on Oct 17, 400 may be too large as a mex len for the ADE corpus.
The result would be many zero-vectors diluting the meaning of each document.
A solution wouldbe to pick a smaller value of maxlen.
"""
maxlen = 100
batch_size = 32         # How many samples to show the net before backpropogating the error and updating the weights
embedding_dims = 300    # Length of the token vectors we will create for passing into the Convnet
filters = 250           # Number of filters we will train
kernel_size = 3         # The width of the filters, actual filters will each be a matrix of weights of size: embedding_dims x kernel_size or 50 x 3 in our case
hidden_dims = 250       # Number of neurons in the plain feed forward net at the end of the chain
epochs = 2              # Number of times we will pass the entire training dataset through the network


def pad_trunc(data, maxlen):
    """ For a given dataset pad with zero vectors or truncate to maxlen """
    new_data = []

    # Create a vector of 0's the length of our word vectors
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)

    for sample in data:

        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data


x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)

x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)



model = Sequential()

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1,
                 input_shape=(maxlen, embedding_dims)))

# we use max pooling:
model.add(GlobalMaxPooling1D())
# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

def makevec(s):
    vl = tokenize_and_vectorize([(1, s)])
    tvl = pad_trunc(vl, maxlen)
    tv = np.reshape(tvl, (len(tvl), maxlen, embedding_dims))
    return tv




# try running some predictions

model.predict(makevec("Patient reports rash and itching after starting amoxicillin."))

model.predict(makevec("Blood pressure stabilized with current lisinopril regimen."))

model.predict(makevec("The old Dune movie was terrible and barely worth watching."))

# Look out for these ones-- as we saw on 10/17, the key words are not found in our vectors!
model.predict(makevec("Patient reports akathesia after starting abilify"))

model.predict(makevec("Patient reports after starting"))

model.predict(makevec("Patient reports movement disorder after starting abilify"))





"""
explore document length distribution
"""
import matplotlib.pyplot as plt

from collections import Counter
c = Counter([len(text) for (label, text) in dataset])

# Extracting keys and values from the Counter
labels, values = zip(*c.items())

# Create the bar chart
plt.bar(labels, values)

# Add title and labels
plt.title('Number of documents of each length')
plt.xlabel('Num tokens')
plt.ylabel('Num documents')

# Show the plot
plt.show()

: 

In [11]:
%pip install huggingface_hub


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
